In [ ]:
# all discrimination on real device using UnifiedTester

from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.backend('ibm_brisbane')
#backend = service.least_busy(15)
 
from ExperimentCircuits import *
from UnifiedTester import *

def generate_hybrid_circuit_4_var(n_qubits:int, n_copies:int):
    ''' 
    generates 4 circuits with both measurements and both types of gates Q<n_qubits>_L<m_layers>_{RZ,IDENT}_{SHORT,XOR}
    '''
    n_layers = n_copies // n_qubits

    circ_id_short = ExperimentCircuits(n_qubits)
    circ_id_xor = ExperimentCircuits(n_qubits)
    circ_rz_short = ExperimentCircuits(n_qubits)
    circ_rz_xor = ExperimentCircuits(n_qubits)

    circ_id_short.set_disc()
    circ_id_xor.set_disc()
    circ_rz_short.set_disc()
    circ_rz_xor.set_disc()

    for _ in range(n_layers-1):
        circ_id_short.qc.barrier()

        circ_id_xor.qc.barrier()

        circ_rz_short.qc.rz(np.pi/n_copies, range(n_qubits))
        circ_rz_short.qc.barrier()

        circ_rz_xor.qc.rz(np.pi/n_copies, range(n_qubits))
        circ_rz_xor.qc.barrier()

    circ_rz_short.qc.rz(np.pi/n_copies, range(n_qubits))
    circ_rz_xor.qc.rz(np.pi/n_copies, range(n_qubits))

    circ_id_short.set_simple_premeas_rot_mtx(True)
    circ_id_xor.set_XOR_premeas_rot_mtx(True)
    circ_rz_short.set_simple_premeas_rot_mtx(True)
    circ_rz_xor.set_XOR_premeas_rot_mtx(True)

    circuits = [circ_id_short.qc, circ_id_xor.qc, circ_rz_short.qc, circ_rz_xor.qc]

    Q = str(n_qubits)
    L = str(n_layers)

    circ_names = ["Q{}_L{}_IDENT_SHORT".format(Q,L), 
                  "Q{}_L{}_IDENT_XOR".format(Q,L), 
                  "Q{}_L{}_RZ_SHORT".format(Q,L), 
                  "Q{}_L{}_RZ_XOR".format(Q,L)]
    
    return circuits, circ_names


circuits = []
circuit_names = []

# prepare sequantial 1-12 copies
for i in range(12):
    tmp_c, tmp_n = generate_hybrid_circuit_4_var(1, i+1)
    circuits.extend(tmp_c)
    circuit_names.extend(tmp_n)

# prepare parallel 1-12 copies
for i in range(12):
    tmp_c, tmp_n = generate_hybrid_circuit_4_var(i+1, i+1)
    circuits.extend(tmp_c)
    circuit_names.extend(tmp_n)

# prepare hybrid 120 copies
for i in [1,2,3,4,5,6,8,10,12]:
    tmp_c, tmp_n = generate_hybrid_circuit_4_var(i, 120)
    circuits.extend(tmp_c)
    circuit_names.extend(tmp_n)

# prepare hybrid 240 copies
for i in [1,2,3,4,5,6,8,10,12]:
    tmp_c, tmp_n = generate_hybrid_circuit_4_var(i, 240)
    circuits.extend(tmp_c)
    circuit_names.extend(tmp_n)

# prepare hybrid 1200 copies
for i in [1,2,3,4,5,6,8,10,12]:
    tmp_c, tmp_n = generate_hybrid_circuit_4_var(i, 1200)
    circuits.extend(tmp_c)
    circuit_names.extend(tmp_n)


print(circuit_names)
utester = UnifiedTester(circuits, backend, optimization_level=0, circuit_names=circuit_names, sim_shots=100000)
print(utester.sim_counts)

['Q1_L1_IDENT_SHORT', 'Q1_L1_IDENT_XOR', 'Q1_L1_RZ_SHORT', 'Q1_L1_RZ_XOR', 'Q1_L2_IDENT_SHORT', 'Q1_L2_IDENT_XOR', 'Q1_L2_RZ_SHORT', 'Q1_L2_RZ_XOR', 'Q1_L3_IDENT_SHORT', 'Q1_L3_IDENT_XOR', 'Q1_L3_RZ_SHORT', 'Q1_L3_RZ_XOR', 'Q1_L4_IDENT_SHORT', 'Q1_L4_IDENT_XOR', 'Q1_L4_RZ_SHORT', 'Q1_L4_RZ_XOR', 'Q1_L5_IDENT_SHORT', 'Q1_L5_IDENT_XOR', 'Q1_L5_RZ_SHORT', 'Q1_L5_RZ_XOR', 'Q1_L6_IDENT_SHORT', 'Q1_L6_IDENT_XOR', 'Q1_L6_RZ_SHORT', 'Q1_L6_RZ_XOR', 'Q1_L7_IDENT_SHORT', 'Q1_L7_IDENT_XOR', 'Q1_L7_RZ_SHORT', 'Q1_L7_RZ_XOR', 'Q1_L8_IDENT_SHORT', 'Q1_L8_IDENT_XOR', 'Q1_L8_RZ_SHORT', 'Q1_L8_RZ_XOR', 'Q1_L9_IDENT_SHORT', 'Q1_L9_IDENT_XOR', 'Q1_L9_RZ_SHORT', 'Q1_L9_RZ_XOR', 'Q1_L10_IDENT_SHORT', 'Q1_L10_IDENT_XOR', 'Q1_L10_RZ_SHORT', 'Q1_L10_RZ_XOR', 'Q1_L11_IDENT_SHORT', 'Q1_L11_IDENT_XOR', 'Q1_L11_RZ_SHORT', 'Q1_L11_RZ_XOR', 'Q1_L12_IDENT_SHORT', 'Q1_L12_IDENT_XOR', 'Q1_L12_RZ_SHORT', 'Q1_L12_RZ_XOR', 'Q1_L1_IDENT_SHORT', 'Q1_L1_IDENT_XOR', 'Q1_L1_RZ_SHORT', 'Q1_L1_RZ_XOR', 'Q2_L1_IDENT_SHORT', 'Q2_

In [2]:
utester.run_job(shots=10000)
utester.job_status()

>>> Job ID: d0bszpyrxz8g008ajnmg
Job running


In [ ]:
# after job finishes (use ibm quantum to check)

utester.collect_counts_from_job()
#utester.collect_counts_from_ext_job(service.job('d0bszpyrxz8g008ajnmg'))

result_data = TesterResultStorage.from_unified_tester(utester)
result_data.save_to_directory("./IBM_BRISBANE_4_5")


In [ ]:
# than u can just load data without using unified tester

from UnifiedTester import *

result_data = TesterResultStorage.load_from_directory("./IBM_BRISBANE")
print(result_data.names[0])
print(result_data.simulated_counts[0])
print(result_data.real_counts[0])

Q1_IDENT_L1_SHORT
{'1': 10000}
{'1': 9586, '0': 414}
